In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cudf
pd.set_option('display.max_columns', 150)

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import gc
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore") # 忽略警告訊息

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
#def objective(trial):    #SET optuna 
#    import optuna
#    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25)
#    dtrain = xgb.DMatrix(train_x, label=train_y)
#    dvalid = xgb.DMatrix(valid_x, label=valid_y)
#   objective_list_reg = ['reg:linear', 'reg:gamma', 'reg:tweedie']
#    metric_list = ['rmse']
#    params ={'boosting':trial.suggest_categorical('boosting', 'gbtree'),
#            'tree_method':trial.suggest_categorical('tree_method', 'gpu_hist'),
#            'max_depth':trial.suggest_int('max_depth', 2, 25),
#            'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
#            'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
#            'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
#            'gamma':trial.suggest_int('gamma', 0, 5),
#            'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
#            'eval_metric':trial.suggest_categorical('eval_metric', metric_list),
#            'objective':trial.suggest_categorical('objective', objective_list_reg),
#            'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree', 0.1, 1, 0.01),
#            'colsample_bynode':trial.suggest_discrete_uniform('colsample_bynode', 0.1, 1, 0.01),
#            'colsample_bylevel':trial.suggest_discrete_uniform('colsample_bylevel', 0.1, 1, 0.01),
#            'subsample':trial.suggest_discrete_uniform('subsample', 0.5, 1, 0.05),
#            'nthread' : -1  
#     }
#
#    bst = xgb.train(param, dtrain)
#    preds = bst.predict(dvalid)
#    pred_labels = np.rint(preds)
#    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
#    return accuracy

In [ ]:
def XGB(train_x, train_y):
    import xgboost as xgb
    model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=11,
    min_child_weight=9.15,
    gamma=0.59,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    alpha=10.4,
    nthread=5,
    missing=-999,
    random_state=2020,
    tree_method='gpu_hist')  # THE MAGICAL PARAMETER)
    model.fit(train_x, train_y)     # 訓練模型
    print("XGBoost version:", xgb.__version__) # just run once
    return model

In [ ]:
def Fillnan(X):
    # fillna
    col = {'feature_7': np.product(X['feature_7'].mode()),'feature_8': np.product(X['feature_8'].mode()),'feature_72': np.product(X['feature_72'].mode()),'feature_78': np.product(X['feature_78'].mode())}
    #col2 = {'feature_11': np.product(X['feature_11'].mode()),'feature_12': np.product(X['feature_12'].mode()),'feature_13': np.product(X['feature_13'].mode()),'feature_14': np.product(X['feature_14'].mode()),'feature_74': np.product(X['feature_74'].mode()),'feature_75': np.product(X['feature_75'].mode()),'feature_80': np.product(X['feature_80'].mode()),'feature_81': np.product(X['feature_81'].mode())}
    col2 = {'feature_74': np.product(X['feature_74'].mode()),'feature_75': np.product(X['feature_75'].mode()),'feature_80': np.product(X['feature_80'].mode()),'feature_81': np.product(X['feature_81'].mode())}
    X.fillna(value=col,inplace=True)
    X.fillna(value=col2,inplace=True)
    del col,col2
    return X

In [ ]:
def Average(X):
    # feature avg
    num = 0
    i = 0
    for feature in X:
        #if (i != 7) and (i != 8) and (i != 72) and (i != 78):
        if (i != 7) and (i != 8) and (i != 72) and (i != 78) and (i != 74) and (i != 75) and (i != 80) and (i != 81):
        #if (i != 7) and (i != 8) and (i != 72) and (i != 78) and (i != 11) and (i != 12) and (i != 13) and (i != 14) and (i != 74) and (i != 75) and (i != 80) and (i != 81):
            indx1 = X[feature] > 0
            indx2 = indx1 == False
            train[feature][indx1] =  1
            train[feature][indx2] = -1
            #train[feature] = X[feature].apply(lambda x:1 if x > 0 else -1) # 以X為條件將train[feature]的Data分兩類
            check = train[[feature, 'resp']].groupby([feature], as_index=False).mean().sort_values(by='resp', ascending=False) #1,-1分群
            if check['resp'][0] > check['resp'][1]:
                num += check['resp'][0] - check['resp'][1] # 求出所有feature差和
            else:
                num += check['resp'][1] - check['resp'][0]
            del check
        i+=1
    #avg = num/126
    avg = num/122
    #avg = num/118  
    return avg
    #0.0005615499604115825
    #
    #0.0005621298415324371

In [ ]:
def Feature(avg,X):
    val = 0
    i = 0
    needft = []
    for feature in X:
        #if (i != 7) and (i != 8) and (i != 72) and (i != 78):
        if (i != 7) and (i != 8) and (i != 72) and (i != 78) and (i != 74) and (i != 75) and (i != 80) and (i != 81):
        #if (i != 7) and (i != 8) and (i != 72) and (i != 78) and (i != 11) and (i != 12) and (i != 13) and (i != 14) and (i != 74) and (i != 75) and (i != 80) and (i != 81):
            indx1 = X[feature] > 0
            indx2 = indx1 == False
            train[feature][indx1] =  1
            train[feature][indx2] = -1
            #train[feature] = X[feature].apply(lambda x:1 if x > 0 else -1) # 以X為條件將train[feature]的Data分兩類
            check = train[[feature, 'resp']].groupby([feature], as_index=False).mean().sort_values(by='resp', ascending=False) # 求兩類平均resp
            if check['resp'][0] > check['resp'][1]: # 大-小
                val = check['resp'][0] - check['resp'][1]
            else:
                val = check['resp'][1] - check['resp'][0]
            if (val - avg) > 0:    # 差 - 平均差
                #X[feature] = X[feature].apply(lambda x:1 if x > 0 else -1)
                needft.append(feature)
            del check,val
        i+=1
    del i
    for feature in needft: # 只做val - avg > 0的data
        indx1 = X[feature] > 0
        indx2 = indx1 == False
        X[feature][indx1] =  1
        X[feature][indx2] = -1
    del indx1,indx2,needft
    return X

In [ ]:
%%time
# load Data
train_cudf  = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

In [ ]:
# draw line chart
fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train['resp']).cumsum()
resp_1= pd.Series(train['resp_1']).cumsum()
resp_2= pd.Series(train['resp_2']).cumsum()
resp_3= pd.Series(train['resp_3']).cumsum()
resp_4= pd.Series(train['resp_4']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative resp and time horizons 1, 2, 3, 4", fontsize=18)
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)
plt.legend(loc="upper left");
del resp_1
del resp_2
del resp_3
del resp_4
gc.collect();

In [ ]:
plt.figure(figsize = (12,5))
ax = sns.distplot(train['feature_117'],
                 bins=3000,
                 kde_kws={"clip":(-5,5)},
                 hist_kws={"range":(-5,5)},
                 color='darkcyan',
                 kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the resp values", size=14)
plt.show();
del values
gc.collect();

In [ ]:
# check
check = train.copy()
check = check.fillna(value={'feature_2': np.product(train['feature_2'].mode())})
check['feature_2'] = check['feature_2'].apply(lambda x:1 if x<-3 else -1)
check = check[['feature_2', 'resp']].groupby(['feature_2'], as_index=False).mean().sort_values(by='resp', ascending=False) #1,-1分群
ck = train['feature_84'].mean()
print(check)

tag = features
new_tag = []
draw = []
for tg in tag:
    if (tag[tg][108] == True) and (tag[tg][114] == True) and (tag[tg][121] == True):
        new_tag.append(tg)
        print(tg)
for tg in new_tag:
    view = tag[tg] == True
    print('tag is ' + tg)
    print(tag['feature'][view])
    draw.append(tag['feature'][view])

del ck,check,tag,new_tag

In [ ]:
# draw missing value 
miss = train.isnull().sum()
px.bar(miss, color=miss.values, title="Total number of missing values for each column").show()
del miss

In [ ]:
# missing value%
missing_values_count = train.isnull().sum()
print (missing_values_count)
total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()
m = missing_values_count['feature_9'].sum()
c = np.product(train['feature_9'].shape)
print("features % = ",m/c*100)
print ("% of missing data = ",(total_missing/total_cells) * 100)
del m,c

In [ ]:
# trainX,labelY
target = train['weight'] != 0 # weight==0無參考價值
train = train[target]
train['action'] = (train['resp'].values > 0).astype('int') # >0才進場投資

X = train.loc[:, train.columns.str.contains('feature')]  # 取出有features的
#X = X.drop(['feature_17','feature_18','feature_27','feature_28','feature_84','feature_90','feature_96','feature_102','feature_108','feature_114'],axis=1)
y = train['action']

del target

In [ ]:
%%time
X = Fillnan(X)
X.fillna(X.mean(),inplace=True)

In [ ]:
print(X)

In [ ]:
%%time
avg = Average(X)
print(avg)

In [ ]:
%%time
X = Feature(avg,X) # 1,-1
del train

In [ ]:
print(X)

In [ ]:
# optuna
#study = optuna.create_study(direction="maximize")
#study.optimize(objective, n_trials=100)

#print("Number of finished trials: ", len(study.trials))
#print("Best trial:")
#trial = study.best_trial

In [ ]:
%%time
model = XGB(X,y)

In [ ]:
for (test_df, prediction_df) in tqdm(iter_test):
    if test_df['weight'].item() > 0: 
        X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
        X_test = Fillnan(X_test)
        #X_test.fillna(-999,inplace=True)
        prediction_df.action = model.predict(X_test)
    else:
        prediction_df.action = 0
    env.predict(prediction_df)